In [ ]:
%matplotlib widget
%reload_ext autoreload
import Observation
import glob, os
package_path = os.path.dirname(os.path.dirname(os.path.abspath(Observation.__file__)))
from Observation import *
import re
from astropy import units as u
import math as mt
from astropy.cosmology import Planck18 as cosmo
from astropy.table import Table, hstack, MaskedColumn#, MaskedConstant



data_path = os.path.join(package_path, "data")
data_cube__path = os.path.join(data_path, "Emission_cube")
if os.path.exists(data_cube__path) is False:
    os.makedirs(data_cube__path)
for name in [ "cube_01.fits", "lya_cube_merged_with_artificial_source_CU_1pc.fits", "CGM_cube.fits", "galaxy_disk_cube.fits", "galaxy_and_cgm_cube.fits"]:
    if os.path.exists(os.path.join(data_cube__path, "cube_01.fits")) is False:
        a = download(
            url="https://nuage.osupytheas.fr/s/fo4AKjoTZ4fBytw/download/"+name,
            file=os.path.join(data_cube__path, name),
        )

# https://docs.google.com/spreadsheets/d/1Es7_ng3crvjEGhgFtbx-YPBEgj2OBgimHJ3YlzlofVg/edit?gid=2066284077#gid=2066284077
instruments, database = load_instruments(sheet_id= "1Ox0uxEm2TfgzYA6ivkTpU4xrmN5vO5kmnUPdCSt73uU", sheet_name= "instruments.csv")

# instruments, database = load_instruments(sheet_id= "1Es7_ng3crvjEGhgFtbx-YPBEgj2OBgimHJ3YlzlofVg", sheet_name= "instruments.csv")

# gaus = lambda x, a, xo, sigma, offset: a ** 2 * np.exp(-np.square((x - xo) / sigma) / 2)+ offset





#  when lamda not good for wavelegnth try to show the issue and change the lambda limit of the sepctra

# [x]  change resolution also for signal not only for atm
# maybe aloow out3 even for slit (just saying that it would take n much more time)

# FB = ExposureTimeCalulator(instrument="SCWI PERF",instruments=instruments,database=database)
# FB = ExposureTimeCalulator(instrument="SCWI Matt",instruments=instruments,database=database)
FB = ExposureTimeCalulator(instrument="SCWI PERF",instruments=instruments,database=database, Δλ=-10,Signal=5e-18,lambda_stack=10, spectra="↳ cube_01-resampled_phys",min=0.54,max=0.85,interpolation="gaussian")
# FB = ExposureTimeCalulator(instrument="SCWI MIN REQ",instruments=instruments,database=database)
# FB = ExposureTimeCalulator(instrument="ASPERA",instruments=instruments,database=database)
# instruments.write("/Users/Vincent/Github/fireball2-etc/notebooks/instruments.csv",overwrite=True)
# FB = ExposureTimeCalulator(instrument="KCWI blue",instruments=instruments,database=database)
# FB = ExposureTimeCalulator(instru/ment="Blue MUSE",instruments=instruments,database=database)
# FB = ExposureTimeCalulator(instrument="CETUS",instruments=instruments,database=database)
# FB = ExposureTimeCalulator(instrument="JUNO-UVS",instruments=instruments,database=database)
# FB = ExposureTimeCalulator(instrument="NISP",instruments=instruments,database=database)
# Hide flux as soon as spectra is selected



# be sure that SNR is right for IFS and slits: 1st plot=real SNR (integrating on several fibers/slicers if IFS), 2nd plot=SNR with only one fiber
# slitwidth and slit length should have different impact on FB1 (because one dimension impacts resolution and mixes with sky)
# clearly with IFS we don't cut flux by the slit base on the PSF at the slit (as it will end in the next slit) - except if the source is << slit?
# then we still integrate sky and flux based on slit size. so it is exactly the same impact as binning: 
# the smaller the fiber, the less we have flux for one fiber but the fllux per arcsec^2 on the image is the same!Is this currently true?

# IFS mode does not impact the SNR when slitdidth>>sigma source. But in the other case, we will loose some flux due to cutting
 # be sure to account well for QE even when throughput curve is given. I guess that I should normalize the throughput curve first and then multiply it by the QE*Atm*Troughput . I think it is fine!
 # Indeed for now, the counts change a lot when we add QE(%) or not. 
 #account for the TH FWHM, for now it is not taken into account. I think it is fine!
 #  change the file system so that when we change the instrument we always to see if there is a throughput curve.
 # Throughput is  not taken into account in the image
# TODO understand why SNR is shit for MUSE narrow... because it's done on a pixel size or element resolution (which is small!) WEIRD IT SMALL EVEN WHE BY SOURCE
# TODO Ajouter la possibilité d'uploader un spectre...
# done TODO for stacking, do not compute N images but just devide the read noise, shot noise etc by sqrt(n)
# TODO understand why the emission line evolution is weird.
# done TODO error with slicer we print a circle because x=y
# DONE TODO be sure to show e-/pix or photons or ADU, etc... Also for the Datacube!
# DONE TODO when we go from imager to FB wi loose the name a tab
# OK: Remove spectra, throughput(λ), Sky lines, atm(λ), delta(λ)
# OK: Reomve Equ width, lambda stack, observed lambda?, Mask PSF (or replace it by sigma ), 
# OK: always remove Spectrograph design (R, slit dim, dispersion, IFS)
# OK: Hide a bunch of parameters in X axis if it is an imager
# OK: Resolve all issues and verify plot update
# OK replace "Spectral image" per "image" when imager (name tab)
# OK Use the Throughput for integrating the band
# OK: use ylog also for the other plot!
# OK: reg;er le problemes de log dans el histogramme et les filtes
# OK:  comprendre pk on voit pas les filtres pour galex
# Find what Image I should show
# What should I add? Magnitude? Or even a spectra?
# instruments to add: HSC
# Different profiles?
# add the magnitude somewhere?
# Put a simulated image from COSMOS or a simulation (cosmic web?), 
# Take all the filters from lephare
# good TODO passage à LUMOS does not work... 
# does not work if I begin with that... FB = ExposureTimeCalulator(instrument="LUMOS")
# OK TODO last subplot show SB limit per pixel and per resolution element and for 2
# TODO ajouter la possibilité d'avoir deux tailles differentes de pixel!! - difficile car il faut changer le type du slider... changer la facon de montrer la slit, etc
# TODO verifier que on peut mettre sky line et atm pour tous les instruments (pour l'isntant ca bug)
# TODO - trouver un moyen pour les imageurs de mettre n'importe quel filtre...
# TODO pour les imageurs avoir la possibilité d'augmenter la taille du filtre pour voir comment ca ameliore les observations
# TODO be sure that SNR per spaxel makes sense for IFS!
# TODO pb for fiber spectro where the spaxel should be the same on both direction 
# Slicer does not work for MUSE
# the flux change when we change source from cube to not

# TODO be sure that the number of pixels is fine! There is a big!! difference between calculating the total flux and dividing by the number of pixels (therefore the PSF grows with number of pixels) and conputing the total flux (which can be hard) and then accountign for the total size and dividing the flux by the number of pixels. 
# So we need to make sure that computing the total flux and the flux per pixel gives the same result!! for an emission line it is kind of easy but more difficult when it is a spectrum.
  
# ca marche mais maintrnant il faut eviter que ca diverge avec pixel scale!
# de meme quand la dispersion augmente normalement quand elle devient plus important que la largeur de la raie!
# TODO should measure the resolution for diffuse source (that taks into account the slit width!)
#  understand why there is a factor 1.5 between fit and sigma source

# understand why the spectra do not overlap!! juste parce que quand je somme sur plusieurs pixels je diminue l'impact du continuum. donc il faut que je prenne les 10 deniers images de chaque extremité du cubbe, que j'interpole lineairement entre chaque pixel entre les deux, puis que je soustraie cela.
# Soit je me laisse une optino soit je commente juste cette ligne ou pas. Normalement ca ne doit jamais rien changer (sauf potentiellement pour le cosmic web qui est entendu en longueur donde)
# take only emission from CGM, 
# subtract the continuum
# fabrit perrot interferometer
# check spectra in annulus
# change size of the stack 
# adaptative smoothing to implement ? impossible !
# TODO quand dispersion diminue vraiment beaucoup, on summ vraiment que sur 
#TODO  ajouter disperson dans titre car des fois il disparait
# weirdly the Throughput_FWHM size in so the same on the plot

#exposure time
# Spectral_resolution
# dispersion


# pixel_scale
# Slitwidth
# EM_gain
# Bandwidth
#---


ValueError: cannot reshape array of size 0 into shape (0,newaxis)

In [8]:
# gaus = lambda  x, a, sigma : a ** 2 * np.exp(-np.square(x / sigma) / 2)
# popt, pcov = curve_fit(gaus, rmean[:40],profile[:40], p0=[np.ptp(profile[:40]), 2])
# self.self.rp_fit[0].set_data(np.linspace(0,np.max(rmean[:40]),100),gaus(np.linspace(0,np.max(rmean[:40]),100),*popt))

# TEST
# un dernier probleme avec la slitwidth, ya le sky qui augmente à l'infini, peut etre uniquement opur les IFS? oui en effet!!! et sinon la dispersion toujorus!!
# pour les IFS, quand quand la source est plus petite que la fente c'est normale que le SNR max soit quand la fente=source
# ndonc dans le caclul du flux total pour les IFS je dois prendre en compte la fente
# import os, sys
# instrument="FIREBall-2 2025"
# # for instrument in instruments.colnames[3::2]:
# FB = ExposureTimeCalulator(instrument=instrument)
# # time.sleep(2)
# SNR_res = "per Res elem"
# x_axis="exposure_time"
# def save(fig, path):
#     # create the folders if they do not exist!
#     os.makedirs(os.path.dirname(path), exist_ok=True)
#     fig.savefig(path, dpi=300, bbox_inches="tight")
# for instrument in ["SCWI Matt"]:#instruments.colnames[4:5]: #"FIREBall-2 2025"
#     FB.instrument.value = instrument  
#     for SNR_res in ["per Res elem","per pix","per Source"]: 
#     # for SNR_res in ["per Res elem"]: 
#         FB.SNR_res.value = SNR_res     
#         for x_axis in ["EM_gain","extra_background","CIC_charge", "QE","RN","Dark_current" ,"cosmic_ray_loss_per_sec", "Signal",'Sky',"Size_source","Line_width", "Atmosphere",'exposure_time','acquisition_time',"readout_time","lambda_stack",  "wavelength", 'Collecting_area',"pixel_scale","Throughput","PSF_RMS_mask","PSF_RMS_det","Spectral_resolution","Slitwidth","Slitlength","dispersion"]:
#         # for x_axis in ["Size_source","dispersion", "pixel_scale","Slitwidth"]:
#             print(x_axis)
#             FB.x_axis.value = x_axis
#             # time.sleep(1)
#             save(FB.fig, "/Users/Vincent/Nextcloud/LAM_new_account/Work/ETC/instruments2/%s/%s_%s_matt.png"%(instrument.replace(" ","_"),x_axis.replace(" ","_"),SNR_res.replace(" ","_")))
#             FB.test.value=False
#             save(FB.fig, "/Users/Vincent/Nextcloud/LAM_new_account/Work/ETC/instruments2/%s/%s_%s.png"%(instrument.replace(" ","_"),x_axis.replace(" ","_"),SNR_res.replace(" ","_")))
#             FB.test.value=True

# import os, sys
# instrument="FIREBall-2 2025"
# # for instrument in instruments.colnames[3::2]:
# FB = ExposureTimeCalulator(instrument=instrument)
# # time.sleep(2)
# SNR_res = "per Res elem"
# x_axis="exposure_time"
# def save(fig, path):
#     # create the folders if they do not exist!
#     os.makedirs(os.path.dirname(path), exist_ok=True)
#     fig.savefig(path, dpi=300, bbox_inches="tight")
# for instrument in instruments.colnames[4:]:
#     FB.instrument.value = instrument  
#     for SNR_res in ["per Res elem","per pix","per Source"]: 
#         FB.SNR_res.value = SNR_res     
#         for x_axis in ["QE","RN","Dark_current" ,"cosmic_ray_loss_per_sec", "Signal","Redshift",'Sky',"PSF_source","Line_width", "Atmosphere",'exposure_time','acquisition_time',"readout_time","lambda_stack",  "wavelength", 'Collecting_area',"pixel_scale","Throughput","PSF_RMS_mask","PSF_RMS_det","Spectral_resolution","Slitwidth","Slitlength","dispersion"]:
#             FB.x_axis.value = x_axis
#             # time.sleep(1)
#             save(FB.fig, "/tmp/%s/%s/%s.png"%(instrument.replace(" ","_"),SNR_res.replace(" ","_"),x_axis.replace(" ","_")))
       
# FB.output_tabs.selected_index = 1
# for instrument in instruments.colnames[3:]:
#     for flux in [1e-17,1e-16,1e-15,1e-14]:
#         FB.Signal.value = flux
#         save(FB.fig2, "/tmp/%s/%s/Flux_%s.png"%(instrument.replace(" ","_"),"Image",flux))
#     FB.instrument.value = instrument     
#     # plt.close()

In [7]:
# FB = ExposureTimeCalulator(instrument="SCWI SPEC",instruments=instruments,database=database, Δλ=-10,Signal=5e-18,lambda_stack=10,min=0.54,max=0.85,interpolation="gaussian") #, spectra="↳ cube_01-resampled_phys"

FB = ExposureTimeCalulator(instrument="SCWI SPEC",instruments=instruments,database=database,Signal=5e-17,lambda_stack=10,min=0.54,max=0.85,interpolation="gaussian") #, spectra="↳ cube_01-resampled_phys"  , Δλ=-1


# def download(url, file=""):
#     """Download a file
#     """
#     from tqdm import tqdm  # , tqdm_gui
#     import requests

#     try:
#         response = requests.get(url, stream=True)
#     except requests.exceptions.RequestException as e:
#         verboseprint(e)
#         return False
#     else:
#         total_size_in_bytes = int(response.headers.get("content-length", 0))
#         block_size = 1024  # 1 Kibibyte
#         progress_bar = tqdm(
#             total=0.95 * total_size_in_bytes, unit="iB", unit_scale=True
#         )
#         with open(file, "wb") as file:
#             for data in response.iter_content(block_size):
#                 progress_bar.update(len(data))
#                 file.write(data)
#         # progress_bar.close()
#         # tqdm_gui.close(progress_bar)
#         # progress_bar.display()
#         # progress_bar.plt.close(progress_bar.fig)
#         # plt.show(block=False)
#         # plt.close('all')
#         # plt.close(progress_bar.fig)
#         if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
#             verboseprint("ERROR, something went wrong")
#             return False
#         else:
#             return True



#exposure time
# Spectral_resolution
# dispersion
# slice width


# pixel_scale
# Slitwidth
# EM_gain
# Bandwidth
#---

In [2]:


# # FIREBall = ExposureTimeCalulator(instrument="FIREBall-2 2023")
# FIREBall.counting_mode
# from Observation import *
# Observation(exposure_time=np.linspace(50,1500,50),RN=50,smearing=0.2,counting_mode=True,plot_=False).compute_optimal_threshold(plot_=True,flux=0.03,size= (int(1e3),int(1e3)))
FB = ExposureTimeCalulator(instrument="SCWI EMCCD",instruments=instruments,database=database, Δλ=-10,Signal=5e-18,lambda_stack=10, spectra="↳ cube_01-resampled_phys",min=0.54,max=0.85,interpolation="gaussian",sky_lines=False,atmlambda=False,QElambda=False)


In [3]:
FB = ExposureTimeCalulator(instrument="SCWI MCP",instruments=instruments,database=database, Δλ=-10,Signal=5e-18,lambda_stack=10, spectra="↳ cube_01-resampled_phys",min=0.54,max=0.85,interpolation="gaussian",sky_lines=False,atmlambda=False,QElambda=False)


In [4]:
FB = ExposureTimeCalulator(instrument="SCWI Skipper",instruments=instruments,database=database, Δλ=-10,Signal=5e-18,lambda_stack=10, spectra="↳ cube_01-resampled_phys",min=0.54,max=0.85,interpolation="gaussian",sky_lines=False,atmlambda=False,QElambda=False)


In [ ]:
# def generate_spiral_galaxy(amplitude,redshift, platescale, PSF_RMS,size_kpc):
#     """
#     Generates a simulated galaxy image of defined size and PSF.
    
#     Parameters:
#     redshift (float): Redshift of the galaxy.
#     platescale (float): Size of one pixel in arcsec/pix.
#     PSF_RMS (float): RMS size of the PSF in arcsec.
    
#     Returns:
#     np.ndarray: Simulated and convolved 2D galaxy image.
#     """
#     # size_kpc = 10.0  # Physical size of the galaxy in kpc
#     intensity = 0.4
#     final_image_size = [500,100]  # Number of pixels per side of the output image
#     angular_size_arcsec = size_kpc / cosmo.kpc_proper_per_arcmin(redshift).to(u.kpc / u.arcsec).value
#     angular_size_pix = angular_size_arcsec / platescale

#     # field_of_view_arcsec = 
#     x = np.linspace(-platescale * final_image_size[0] / 2, platescale * final_image_size[0] / 2, final_image_size[0])
#     y = np.linspace(-platescale * final_image_size[1] / 2, platescale * final_image_size[1] / 2, final_image_size[1])
#     xx, yy = np.meshgrid(x, y)
#     r = np.sqrt(xx**2 + yy**2)
#     theta = np.arctan2(yy, xx)
    
#     core = np.exp(-r**2 / (2 * (angular_size_arcsec / 5 / 2.355)**2))
#     spiral = np.exp(-r**2 / (2 * (angular_size_arcsec / 2.355)**2)) * (1 + intensity * np.sin(2 * theta + r * 20 / angular_size_arcsec))
#     galaxy = core + spiral

#     # Normalize the galaxy to conserve total energy
#     galaxy_sum_initial = np.max(galaxy)*100
#     galaxy *= amplitude /galaxy_sum_initial  # Normalize so that the sum is 1
#     galaxy *= size_kpc * size_kpc  # Scale to maintain the total energy constant

#     PSF_RMS_pix = np.sqrt(PSF_RMS**2 + 0.05**2) / platescale
#     galaxy_convolved = gaussian_filter(galaxy, PSF_RMS_pix)

#     return galaxy_convolved

# # plt.figure();plt.imshow(generate_spiral_galaxy(amplitude=FB.flux, redshift=0.001, platescale=FB.pixel_scale.value, PSF_RMS=FB.PSF_RMS_det, size_kpc=12));plt.show()

In [ ]:
# (os.path.exists("../data/Instruments/%s/Sky_emission_lines.csv"%(FB.instrument.replace(" ","_"))) )
# print(1)
# a = Table.read("Sky_emission_lines/spectra.csv")
# fig = plt.figure(figsize=(15,3))
# plt.loglog(a["# wavelength"]/10000,a["emission_lines"],":",c="k",alpha=0.8,lw=0.2)
# plt.ylim(ymin=1e-2)
# plt.xlabel("Wavelength (microns)")
# plt.ylabel("Sky flux in \n1E-16 erg/[s A cm**2 arcs**2]")
# fig.tight_layout()
# plt.show()
# # sky model
# # sky observing mode
# # different modes
# # seing

In [ ]:
# import numpy as np






# data = """
# -27.09946824154065,   0.5656884604299096
# -17.763888116112412, 1.1948670570576576
# -9.97430231541449,   2.43555498224965
# -0.6426964684559948, 4.964508843492912
# 8.68096082156302, 9.423816134476512
# 21.104555317982204, 20.626853767154984
# 38.17805562400167, 55.90139946933155
# 59.885564625742404, 162.6822634252464
# 81.57717651360417, 410.58403825329606
# 112.55270290678749, 1377.7653482012001
# -0.539365228242616, 12.529627053302825
# -0.45987965884773985, 25.53974135879882
# -0.3883426463923456, 48.48049107675056
# -0.3009085200579875, 106.1141246529154
# -0.14988593820771712, 410.5840382532977
# -0.08629748269180482, 725.8129995108268
# """

# # Split the data into lines and then into individual values
# lines = data.strip().split('\n')
# Temp_c = []
# Pressure_mbar = []

# for line in lines:
#     temp, pressure = map(float, line.split(','))
#     Temp_c.append(temp)
#     Pressure_mbar.append(pressure)

# # Convert lists to numpy arrays
# Temp_c = np.array(Temp_c)
# Pressure_mbar = np.array(Pressure_mbar)

# print("Temp_c:", Temp_c)
# print("Pressure_mbar:", Pressure_mbar)



# import numpy as np
# import matplotlib.pyplot as plt

# # Définition de la fonction de pression de vapeur selon la formule de Buck
# def buck_pressure(T):
#     """Calcule la pression de vapeur d'eau en kPa pour une température donnée (T en °C)."""
#     return 0.61121 * np.exp((18.678 - (T / 234.5)) * (T / (257.14 + T)))

# # Définition des plages de température
# T_range = np.linspace(-10, 15, 200)  # Température de -10°C à 15°C

# # Calcul des pressions en mbar (1 kPa = 10 mbar)
# P_range = buck_pressure(T_range) * 10

# # Création du graphique
# fig, ax = plt.subplots(figsize=(10, 5))

# # Tracé de la courbe
# ax.plot(T_range, P_range)#, label="Pression de vapeur de l'eau (Buck)", color="blue")

# # Fill between to show the 3 phases of water
# ax.fill_between(T_range, 0, P_range, where=(T_range <= 100), color='lightcoral', alpha=0.1, label="Solid (Ice)")
# # ax.fill_between(T_range, 0, P_range, where=(T_range > 0), color='lightgreen', alpha=0.3, label="Liquid (Water)")
# ax.fill_between(T_range, P_range, 18, where=(T_range <= 0), color='lightblue', alpha=0.1, label="Vapor (Steam)")
# ax.fill_between(T_range, P_range, 18, where=(T_range > 0), color='lightyellow', alpha=0.1, label="Vapor (Steam)")

# # Ajout des annotations importantes
# # ax.axhline(y=7.5, color="deepskyblue", linestyle="--", label="Pression nominale du Dewar (~7-8 mbar)")
# # ax.axvline(x=2.4, color="blue", linestyle="--", label="Température d'évaporation nominale (~2.4°C)")

# # Zone critique de gel de l'eau (remplissage gris)
# # ax.fill_between(T_range, 0, 4, color='gray', alpha=0.3, label="Zone critique de gel de l'eau")

# # Ajout de texte explicatif
# # ax.annotate("Évolution de T/P en vol", xy=(5, 10), xytext=(7, 14),
# #             arrowprops=dict(facecolor='black', arrowstyle="->"), fontsize=10)

# # Labels et titre
# ax.set_xlabel("Temperature (°C)")
# ax.set_ylabel("Pressure (hPa)")
# ax.set_title("Temperature Dependence - Evaporation Pressure of Water")

# # Légende
# # ax.legend()
# plt.xlim(-11,16)
# plt.ylim(2,18)
# # plt.yscale("log")
# # plt.plot(Temp_c,Pressure_mbar)
# # Affichage du graphique
# plt.grid(True, linestyle="--", alpha=0.5)
# plt.show()




<!-- # <center>FIREBall-2 SNR calculator </center>

### Explanation
- Gives the estimated signal-to-noise ratio for an extended source on a resolution element
- **Variables:** Source's flux, sky, aquisition time, exposure time, EM gain, dark current, CIC, readnoise, readout time, thresholding, smearing exponential length and temperature if you check it (based on a first rough evolution of smearing and dark current with temperature, therefore changing the temperature will change smearing and dark accordingly.)
- **Fixed Values:** QE = 45%, resolution element=57 mu, pixel_size = 13 mu, throughput = 13%, atmosphere transmission = 50%
- **Outputs:** In the top pannel, the noise of the different sources (Signal, Dark, Sky, CIC, RN) is given in e- per resolution element per N frames on the total acquisition time. The middle pannel we give the average e-/pix value for each component (before stacking). We give the relative fraction of all sources of noise and the resulting SNR. When thresholding is applied, the smearing can be changed as it will impact the position of the threshold that optimizes the SNR. For this optimal threshold, the fraction of signal and readnoise kept (above the threshold) is given.
- **Caution:**  it appears that for EMCCDs the dependencies are pretty complex/non-linear. In particular, the lack of models on the impact of amplification, thresholding, and register clearing... combined with smearing on the final SNR makes risky the fact of relying only on modeling to choose the optimal temperature/gain/etc. That's why I suggest an [end-to-end test](https://docs.google.com/document/d/1SpiEK8MBmYduKUFvWmnTOfkfsX4VWs6JMF5-YJKDELo/edit?usp=sharing) validating this: 


### Analysis
- **Smearing:** At -120C, the low device's CTE smears the pixels' counts and then lowers the effective gain. This lowering factor is huge (>2) for a smearing length of 1.5, but is much smaller for a smearing length of 0.7 (~1.2). 
It seems that we already had a comparabale 0.7 smearing length with previous controller at 10MHz (tests February 7th and 8th 2019) at same EMgain (1600) and temperature (-95C). Which might mean that the 1MHz reading does not decrease significantly the smearing. **With the current (2022) smearing length, thresholding does not increase the SNR!! If we manage to invert the smearing, even if it doubles the readnoise, we could recover most of the thresholding efficiency.**
- **Photon-counting:** The thresholding effect can be misleading as it depends on Flux, EMgain, RN, smearing and noise($\sigma_{CIC}^2 + \sigma_{DC}^2+ \sigma_{SKY}^2$). But actually it depends mostly on the ratio $\frac{G_{EM,eff}}{RN}$. For a ratio >> 10 the thresholding is efficient. As stated above for a smearing length of 0.7, EMgain and effective_gain are close. A consequence of that, is that at the EMgains we use ($\sim1600$) and smearing $\sim 0.7$, the thresholding is almost as efficient with RN=50 and RN=107 as the ratio $\frac{G_{EM,eff}}{RN}$ stays >>10. This was not the case in 2018 when the smearing length was 1.5. This is why in photon counting mode, you will see a very small impact of read noise change.
- **Reading frequency tradeoff:** Based on above, the change from 10MHz to 1MHz as minor consequences on the total SNR.
    - In photon counting mode and with low smearing, the decrease of the RN has quasi no effect (~1%) on the SNR
    - The transition from 1.4 to 3sec readout would mean a loss of SNR of ~2%, transition to 10sec about ~10% 
    - Therefore the real critical point is the possible need to add or not a shutter above the tank with the 1MHz controller as it will complexify guidance
- **Temperature Tradeoff:** Conclusion depends on the definition of SNR on thresholded images (see below): In both cases, the SNR decreases with smearing but in one case smearing has more important impact and compensate  dark current giving rise to an optimal temperature. Though, the fact that we will never get rid of dark current while we could actually manage to inverse the smearing could suggest staying at low temperature (-120 to -110).

 
  
  
|             **Parameters**             |          **2022 1Mhz controller**           | **Values 2018** |
| :------------------------------------: | :-----------------------------------------: | :-------------: |
|          **Temperature (C)**           |                     -95                     |      -115       |
|       **Conversion gain ADU/e-**       |                    1/4.5                    |      0.53       |
|           **Read noise e-**            |            50 -> 60 longer cable            |       107       |
|         **pCIC e− /pix/frame**         |                    0.003                    |      0.004      |
|         **sCIC e− /pix/frame**         |                   ~0.002                    |     ~0.002      |
|  **semi-amplified CIC e− /pix/frame**  |                    ~0.02                    |     ~0.03-4     |
|           **EM gain  e-/e-**           |          O                    2000          |      1400       |
|   **Smearing exponent. length pix**    |              L~0.6 ± 0.1  pix               |   L~1.5 ± 0.1   |
|       **Dark current e− /pix/h**       |                   ~2±0.5                    |   0.5 ± 0.03    |
| **Cosmetics % pixels bottom detector** |                     ~2%                     |   <<1% (TBD)    |
|         **Exposure time sec**          |                     50                      |       50        |
|          **Readout time sec**          | ~10 (all), ~3(physical) +2 better clearance |      ~1.4       |


<!-- 
### Need to add
- Predictions for point source, sources on several resolution element, stack of galaxies
 -->

<!-- Based on the 2019 and 2022 images the consequences of a 20 degrees increase is important. With the first approximations, the increase of the dark current combined with the decrease of the smearing length might lead to a 5% SNR increase in counting mode (+sharper PSF). Though, for now, the major consequence of the temperature increase seems to be the high fraction of cosmetics on the lower part of the image. A significant fraction could be corrected for with appropriated software but the fraction can become extremely high and prevent good correction (needs further implementation). This is not taken into account in this model. Decreasing the tempetrature to -100C (TBD) might be a good trade off (in 2019 we had 0.8 smearing at -95C and EMgain=1600). For very bright objects the shot noise becomes dominant and then the impact of dark current is smaller so the SNR peaks at higher temps (80).
  -->

<!-- 
## Thresholding analysis
The threshold is computed by optimizing the SNR:

$$ SNR_1(T) = F_{True,pos}(T)\frac{S \times F_{e^->0,ADU>T}}{\sqrt{ F_{e^->0,ADU>T} \times(S + \sigma_{CIC}^2 + \sigma_{DC}^2+ \sigma_{SKY}^2)  + F_{e^-=0,ADU>T} \frac{\sigma_{RN}^2}{Gain} }} 
\sim F_{True,pos}(T) \frac{S \sqrt{ F_{e^->0,ADU>T}}}{\sqrt{S + \sigma_{CIC}^2 + \sigma_{DC}^2+ \sigma_{SKY}^2 }}$$



with:
- $\sigma_{RN}^2=\frac{RN}{G_{EM}} \times \frac{\sum RN[RN>T]}{\sum RN} = \frac{RN}{EMgain}  \times \%pix_{e^-=0}^{ADU>T}$: ie 60e-/2000 x the fraction of pixels above the threshold which dit not receive any photo-electrons 
- $ F_{e^->0,ADU>T}$ :  fraction of pixels above the threshold which received more than 0 electons
- $ F_{e^-=0,ADU>T}$ :  fraction of pixels above the threshold which did not receive any electron
- $F_{True,pos} = \frac{F_{e^->0,ADU>T}}{F_{ADU>T}} $ : the fraction of true positives = the fraction of pixels above the threshold which received more than 0 electons. This uncertainty term is not in the final SNR formula as it converges towards ~1 after stacking as the average pixel value will converge towards the real number of incoming photo-electrons

The thresholding depends on 5 factors:
- **The gain**: the higher the gain the easier it is to differentiate pixels who received a photo-electron and those which did not
- **The read noise**: the lower the RN the easier it is to differentiate pixels who received a photo-electron and those which did not
- **The low CTE** smears the charges, from pixels who did receive a photo-electron to pixel who did not and lower the effective gain $G_{EM,eff}$
- **The signal**: because thresholding only allows to recover 0 and 1, thresholding is only efficient at flux <<1e-/exp 
- **The additional noise**: $\sigma = \sigma_{CIC}^2 + \sigma_{DC}^2+ \sigma_{SKY}^2$ has relatively small impact on the impact of the thresholding

**At the end, the thresholding impact is:**
- Highly non linear, many things to learn
- The efficiency of the thresholding depends mostly of the ratio $\frac{G_{EM,eff}}{RN}$, which extremely impacted by the smearing.
- The thresholding SNR gain (SNR$_{threshold}$/SNR$_{analogic}$) lies within $[0,\sqrt{2}]$. Condditions are pretty stricts to get som thresholding efficiency and smearing makes the upper bound falls dramatically. 
- Therefore if might not worth it to loose SNR in order to be in these very conditions as we will loose it anyway in the final budget.
- The thresholding SNR gain seems less impacted by read noise. Which means that if we manage to do some (linear?) inversion of the smearing which increases the noise, it might still increase the thresholding gain above 1. 
- The only real game changer in terms of SNR is the smearing 
- **Fraction kept (photo-electrons above threshold)**
    - Important dependance on smearing and flux, less important one on gain and read noise.
    - At low smearing the bigger the ratio gain/ron the higher the fraction of charges kept.
    - Evolution with smearing is dramatic, higher gain does not help much, lower RN helps
    - Fraction increase with flux 
- **Fraction fake detections (0e- pixels above threshold)**
    - Increases importantly with smearing
    - does not depend much on read noise (small positive correlation)
    - decreases with gain
    - Increases as Flux increase. Not sure to understand why... Maybe because the number of pixels with 0 photo-e- cecreases 
- **Thresholding SNR gain**
    - Thresholding is efficient only if >1
    - Dramatically decreases with smearing (No interest of thrsholding above smearing legth of 0.6)
    - Increases with gain and decreases with read noise if no smearing
    - if smearing, the read noise has no impact.
    - Increases with flux
- **Threshold**
    - increase extremely with smearing
    - Without smearing, poor dependance on gain and readnoise
    - If smearing, increases as readnoise decreases
    - If smearing: Increasing with gain!
    - Increases with flux

 -->

<!-- 

|   **Thresholding dependancy table**                   | **Smearing <br />  ( 0 - 2 )** | **Read noise  <br /> ( 30 - 120 )** | **Gain  <br /> ( 800 - 2500 )** | **Flux <br /> ( 0.01 - 1.5 )** |
|:--------------------:|:------------------:|:-----------------------:|:-------------------:|:------------------:|
|     **Photo-electron Fraction kept**    |    --- <br /> 92% - 66%    |       -  <br /> 72% - 71%       |     .   <br /> 72% - 71%    |     +  <br /> 75% - 76%    |
| **RN fraction kept** |  +++  <br /> 0.6% - 12.4%  |       +  <br /> 0.7% - 1%       |    -  <br /> 1.1% - 0.5%    |    ++?  <br /> 0.3% - 3%   |
|     **Threshold**    |   +++  <br /> 3 σ - 11 σ   |      --  <br /> 18 σ - 5 σ      |     + <br />  5 σ - 16 σ    |    +? <br />  8 σ - 13 σ    |
|        **Thresholding SNR gain**       |     --- <br />  1.3 - 0.6   |        --  <br /> 1.35 - 1.22        |      +  <br /> 1.13 - 1.18      |      ++  <br /> 1.16 - 1.21     |
 -->
